In [ ]:
# Package Setup
import Pkg;
# Pkg.update()

# Julia Packages
using Distributions
using LinearAlgebra
using Statistics
using PGFPlots

# Satellite Dynamics Packages
using SatelliteDynamics

# Load SatelliteTasking - Reclone to keep version current
Pkg.clone("..") # For some reason this doens't work with Pkg.add + PackageSpec. Why?
using SatelliteTasking
using SatelliteTasking.SatellitePlanning
using SatelliteTasking.Analysis

# Temporary for now
Pkg.add("JuMP")
Pkg.add("Gurobi")
using JuMP
using Gurobi

In [ ]:
# Configure simulation
epc0 = Epoch(2019, 1, 1, 0, 0, 0, tsys=:UTC) # Start of time span
epcf = Epoch(2019, 1, 2, 0, 0, 0, tsys=:UTC) # End of simulation time span

# Set Simulation Time Step
timestep = 1
dtmax    = 5

# Define Satellite Orbit
oe   = [R_EARTH + 500e3, 0, 90.0, 0, 0, 0]
eci0 = sOSCtoCART(oe, use_degrees=true)

# Numer of perturbed orbits to simulate
num_orbits = 1

# Set Perturbation Values 
pos_error = 5000 # Position knowledge error [m]
vel_error = 5    # Velocity knowledge error [m/s]
orb_mean  = zeros(Float64, 6)
orb_sdev  = vcat((pos_error/sqrt(3)*ones(Float64, 3))..., (vel_error/sqrt(3)*ones(Float64, 3))...)

# Simulate true and perturbed orbits
@time true_orbit, perturbed_orbits, eci_errors = simulate_orbits(num_orbits, epc0, epcf, eci0, orb_mean, orb_sdev, timestep=timestep, dtmax=dtmax);

In [ ]:
# Compute True and perturbed collects

# Load test images
# @time images = load_images("../data/landsat_test_150.json", dwell_time=5.0);
@time images = load_images("../data/landsat_test_300.json", dwell_time=5.0);
# @time images = load_images("../data/landsat_test_600.json", dwell_time=5.0);
num_images = length(images)

@time true_opportunities, perturbed_opportunities, mean_diff, sdev_diff, missing_opportunities = compute_perturbed_opportunities(true_orbit, perturbed_orbits, images, epc_step=3600);
@time collects = compute_collects_by_number(true_opportunities, 10);

# Compute feasible collects
image_collects = group_image_collects(collects) # Group collects by image
num_feasible   = 0
for img in keys(image_collects)
    if length(image_collects[img]) > 0
        num_feasible += 1
    end
end
pct_feasible = num_feasible/num_images*100

println("$num_feasible out of $num_images images have collection opportunities.")

In [ ]:
# Plot Differences in Opportunities
Axis([
    Plots.Linear(1:24, sdev_diff[1, :], legendentry="Start of Window")
    Plots.Linear(1:24, sdev_diff[2, :], legendentry="End of Window")
    Plots.Linear(1:24, sdev_diff[3, :], legendentry="Window Duration")
    Plots.Linear(1:24, missing_opportunities, legendentry="Number of Missing Opportunities")
], width="10cm", height="10cm", legendPos="north west", xmin=0, xmax=24, ymin=0)

In [ ]:
# Graph planning
@time path, reward, image_list = sp_graph_policy(collects, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $reward")
println("Number of images collected: $(length(image_list))/$num_images, $(length(image_list)/num_images*100)")
println("Number of feasible images collected: $(length(image_list))/$num_feasible, $(length(image_list)/num_feasible*100)")

In [ ]:
# MILP planning
@time path, reward, image_list = sp_milp_policy(collects, Function[constraint_agility_single_axis], horizon=0.0, allow_repeats=false)

println("Total planning reward: $reward")
println("Number of images collected: $(length(image_list))/$num_images, $(length(image_list)/num_images*100)")
println("Number of feasible images collected: $(length(image_list))/$num_feasible, $(length(image_list)/num_feasible*100)")

In [ ]:
# # Mixed-Integer Programming Planning

# mcollects = deepcopy(collects)

# # Initialize MILP problem
# milp = Model(solver=GurobiSolver(Presolve=0, Heuristics=0.0)) # Cuts=0

# # Sort Collects to ensure they are in time-asecnding order
# sort!(mcollects, by = x -> x.sow)

# # constraint_list = Function[constraint_agility_single_axis]
# constraint_list = Function[constraint_agility_single_axis]

# horizon = 0
# allow_repeats = false

# # Initialize Variables
# @variable(milp, x[1:length(mcollects)], Bin)

# # Add Objective
# @objective(milp, Max, sum(col.image.reward*x[i] for (i,col) in enumerate(mcollects)))

# # Define non-repetition constraints if necessary
# if allow_repeats == false
#     # Group collects by image
#     image_collects = group_image_collects(mcollects) # Group collects by image
    
#     # Add constraints to limit one collect per image
#     for img in keys(image_collects)
#         # Only add constraints for when there is more than one possible collect
#         if length(image_collects[img]) > 1
#             @constraint(milp, sum(x[i] for i in collect(e[1] for e in image_collects[img])) <= 1)
#         end
#     end
# end

# # Add satellite model-derived constraints
# for i in 1:length(mcollects)
#     for j in  i:length(mcollects)
#         # Since all constraints are reciprocal they only need to be checked in one direction
#         col_start = mcollects[i]
#         col_end   = mcollects[j]
        
#         # Skip adding constraints if a planning horizon is being used
#         if horizon > 0 && col_end.sow > (col_start.eow + horizon)
#             # Condition to exit early is only considering transitions within a certain horizon may be invalid
#             continue
#         end

#         if col_start == col_end || col_start.image == col_end.image || col_start.opportunity == col_end.opportunity || j < i
#             # Skip if the same opportunity, or same image because this is already covered
#             continue
#         else
#             # Transition is default valid
#             valid_transition = true
            
#             # Only evaluate transitions if time constraint doesn't matter
#             for constraint in constraint_list
#                 # Use logical and to evaulate path feasibility
#                 # Because this is a binary comparison (independent of time), and constraints are nominally
#                 # evaluated with start dependent on the end, we only add a constraint if neither transition is valid
#                 # Otherwise the problem would be over constrained just due to the final opportunity not
#                 # being able to take images before the current
#                 valid_transition = valid_transition && (constraint(col_start, col_end) || constraint(col_end, col_start))
                
#                 # Add constraint as soon as invalid to short-circuit additional evaluations
#                 if !valid_transition
#                     # Boolean logic constraint permitting taking start opportunity, but
#                     @constraint(milp, x[i] + x[j] <= 1)
#                     # println("Invalid due to agility")
#                     continue
#                 end
#             end
#         end
#     end
# end

# # println(milp);

In [ ]:
# status = solve(milp);

In [ ]:
# # count = 0
# # taken = []
# # for i in 1:length(mcollects)
# #     if getvalue(x[i]) != 0.0
# # #         println("x[$i] = $(getvalue(x[i])), Collect: $(string(mcollects[i]))")
# #         count += 1
# #         push!(taken, i)
# #     end
# # end

# # println("Count: $count")
# # println(taken)
# # feasible = []
# # for i in 1:length(taken)-1
# #     push!(feasible, constraint_agility_single_axis(mcollects[taken[i]], mcollects[taken[i+1]]))
# # end

# # println(feasible)

# max_col = 0
# img_max = nothing
# for img in keys(image_collects)
#     if length(image_collects[img]) > max_col
#         max_col = length(image_collects[img])
#         img_max = img
#     end
# end

# println(max_col)
# println(img_max)
# for col in image_collects[img_max]
#     println(col)
# end